In [1]:
from gecommon import *



In [33]:
def get_levels_12():
    body = {"query":{"bool":{"must":[{"term":{"level":{"value":12}}}]}}}
#     al = es.search(index='map_work', size=100000, body=body)
    al = elasticsearch.helpers.scan(es, index='map_work', query=body, scroll='5m', size=1000)
    ret = []
    for i in al:
        ret.append(i['_id'])
    return ret

def gen_level_1_8(levels):
    ret = set()
    for i in levels:
        ret.add(i[:8])
        ret.add(i[:4])
    ret.add('0')
    return list(ret)

def publish_works(levels):
    req = []
    n = 0
    es = elasticsearch.Elasticsearch(['4.4.4.4'])
    for i in levels:
        n += 1
        if n % 1000 == 0:
            print('Got %d rows' % n)
        a = {'levels':i, 'level':len(i), 'status':'waiting', 'worker':'none'}
        b = {'_index':'map_work', '_type':'work', '_id':i, '_source':a}
        req.append(b)
        if len(req) == 1000:
            elasticsearch.helpers.bulk(client=es, actions=req)
            req = []
    if req:
        elasticsearch.helpers.bulk(client=es, actions=req)
    print('Done')
    


In [15]:
body = {"query":{"bool":{"must":[{"term":{"level":{"value":12}}}]}}}
al = elasticsearch.helpers.scan(es, query=body, scroll='5m')
elasticsearch.helpers.scan?

In [12]:
next(al)

{'_id': '021031231201',
 '_index': 'map_work',
 '_score': None,
 '_source': {'level': 12,
  'levels': '021031231201',
  'status': 'waiting',
  'worker': 'none'},
 '_type': 'work',
 'sort': [0]}

In [30]:
c = get_levels_12()

In [31]:
d = gen_level_1_8(c)

In [24]:
len(d)

197

In [34]:
publish_works(d)

Done
